In [1]:
import os
import sys

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from myutils import prep, drop, statusout, batch_gen, seq2sent, index2word
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from custom.graphlayers import GCNLayer
from timeit import default_timer as timer
import h5py
import pickle
from custom.graphlayers import GCNLayer
import model
import numpy as np

## Models

In [ ]:
for curdir, dirs, files in os.walk('../outdir/models'):
    for f in files:
        print(os.path.join(curdir,f))

## Config Stuff

In [4]:
gpu = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
data_path = '/nfs/projects/funcom/data/javastmt/q90'

In [5]:
# Select 2 models
modelfile1 = '../outdir/models/seed_565656/attendgru_E04_1617646710.h5'
modelfile2 = '../outdir/models/seed_1337/attendgru_E07_1617637141.h5'

In [6]:
# Load data
extradata = pickle.load(open('{}/dataset_short.pkl'.format(data_path), 'rb'))
h5data = h5py.File('{}/dataset_seqs.h5'.format(data_path), 'r')

In [7]:
model1 = keras.models.load_model(modelfile1, custom_objects={'tf':tf, 'keras':keras, 'GCNLayer':GCNLayer})
config = {}
config['comvocabsize'] = extradata['comstok'].vocab_size
config['batch_size'] = 200
config['locfid'] = extradata['locfid']
loc2fid = config['locfid']['ctrain']
config['comlen'] = model1.inputs[1].shape[-1]
config['tdatlen'] = model1.inputs[0].shape[-1]
config['batch_config'] = [ ['tdat', 'com'], ['comout'] ]
allfidlocs = list(loc2fid.keys())
batch_sets = [allfidlocs[i:i+config['batch_size']] for i in range(0, len(allfidlocs), config['batch_size'])]

In [8]:
model2 = keras.models.load_model(modelfile2, custom_objects={'tf':tf, 'keras':keras, 'GCNLayer':GCNLayer})

In [12]:
batchsize = 2000
comlen = config['comlen']
epochs=10

model = keras.models.Sequential()
model.add(keras.Input(shape=(config['comvocabsize']*2,)))
#model.add(keras.layers.Dense(10908, activation='relu'))
model.add(keras.layers.Dense(config['comvocabsize'], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

gen1 = batch_gen(h5data, extradata, 'train', config, training=True)
gen12 = batch_gen(h5data, extradata, 'train', config, training=False)

gen2 = batch_gen(h5data, extradata, 'train', config, training=True)
gen22 = batch_gen(h5data, extradata, 'train', config, training=False)

print(model.summary())
for i in range(epochs):
    for n, fidloc_set in enumerate(batch_sets):
        print("Batch {}".format(n))


        (inputs, outputs) = gen1.make_batch(fidloc_set)
        (batch, badfids, comseqpos) = gen12.make_batch(fidloc_set)


        (inputs2, outputs2) = gen2.make_batch(fidloc_set)
        (batch2, badfids2, comseqpos2) = gen22.make_batch(fidloc_set)

        new_out = []
        for i in range(int(len(outputs)/comlen)):
            tmp = outputs[i*comlen:(i+1)*comlen]
            tmp2 = []
            for k in tmp:
                tmp2.append(np.argmax(k))
            new_out.append(tmp2)


        new_out = np.array(new_out)
        comlen = config['comlen']
        fiddats = list(zip(*batch.values()))
        fiddats2 = list(zip(*batch.values()))

        nfiddats = list()

        for fd in fiddats:
            fd = np.array(fd)
            nfiddats.append(fd)


        nfiddats2 = list()
        for fd in fiddats2:
            fd = np.array(fd)
            nfiddats2.append(fd)

        #no = np.zeros((batchsize, comvi))

        for i in range(1, comlen):
            results = model1.predict(nfiddats, batch_size=batchsize)
            results2 = model2.predict(nfiddats2, batch_size=batchsize)

            allsults = np.concatenate((results, results2), axis=1)

            out = outputs[i::13]
            model.fit(allsults,out, epochs=1, verbose = 1, batch_size=200)

            for c, (t, o) in enumerate(zip(results, out)):
                m = np.argmax(t)
                #print(comstok.i2w[np.argmax(o)],'---', comstok.i2w[m])
                nfiddats[comseqpos][c][i] = m

            #print('------------')
            #print(m)

    #     final_data = {}
    #     final_real = {}
    #     for fid, com, real in zip(batch.keys(), nfiddats[comseqpos], new_out):
    #         final_data[fid] = seq2sent(com, comstok)
    #         final_real[fid] = seq2sent(real, comstok)

    #     for key, val in final_data.items():
    #         print(key, final_real[key], val)
    

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10908)             237979836 
Total params: 237,979,836
Trainable params: 237,979,836
Non-trainable params: 0
_________________________________________________________________
None
Batch 0
1/1 [==============================] - 0s 83ms/step - loss: 9.2666 - accuracy: 0.0000e+00
Batch 1
1/1 [==============================] - 0s 83ms/step - loss: 9.2124 - accuracy: 0.0450
Batch 2
1/1 [==============================] - 0s 82ms/step - loss: 9.1756 - accuracy: 0.1000
Batch 3
1/1 [==============================] - 0s 83ms/step - loss: 9.1325 - accuracy: 0.0950
Batch 4
1/1 [==============================] - 0s 82ms/step - loss: 9.0879 - accuracy: 0.1600
Batch 5
1/1 [==============================] - 0s 81ms/step - loss: 9.0369 - accuracy: 0.1800
Batch 6
1/1 [==============================] - 0s 82ms/step -

Exception ignored in: <bound method ScopedTFGraph.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFGraph object at 0x7fa7da08b7b8>>
Traceback (most recent call last):
  File "/home/alex/.virtualenvs/python3.6/lib/python3.6/site-packages/tensorflow/python/framework/c_api_util.py", line 58, in __del__
    self.deleter(self.graph)
KeyboardInterrupt


1/1 [==============================] - 0s 82ms/step - loss: 4.5493 - accuracy: 0.1900


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/alex/.virtualenvs/python3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-1e70100e5c3b>", line 60, in <module>
    results = model1.predict(nfiddats, batch_size=batchsize)
  File "/home/alex/.virtualenvs/python3.6/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1629, in predict
    tmp_batch_outputs = self.predict_function(iterator)
  File "/home/alex/.virtualenvs/python3.6/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 828, in __call__
    result = self._call(*args, **kwds)
  File "/home/alex/.virtualenvs/python3.6/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 862, in _call
    results = self._stateful_fn(*args, **kwds)
  File "/home/alex/.virtualenvs/python3.6/lib/python3.6/site-packages/tensorflow/python/eager/function.py", line 29


KeyboardInterrupt



In [ ]:
#outfile = open('test_ensemble.txt', 'w')
for n, fidloc_set in enumerate(batch_sets):

    gen1 = batch_gen(h5data, extradata, 'test', config, training=True)
    gen12 = batch_gen(h5data, extradata, 'test', config, training=False)
    
    (inputs, outputs) = gen1.make_batch(fidloc_set)
    (batch, badfids, comseqpos) = gen12.make_batch(fidloc_set)
    new_out = []
    for i in range(int(len(outputs)/comlen)):
        tmp = outputs[i*comlen:(i+1)*comlen]
        tmp2 = []
        for k in tmp:
            tmp2.append(np.argmax(k))
        new_out.append(tmp2)

    new_out = np.array(new_out)
    comlen = config['comlen']
    fiddats = list(zip(*batch.values()))

    nfiddats = list()
    
    for fd in fiddats:
        fd = np.array(fd)
        nfiddats.append(fd)
        
    #no = np.zeros((batchsize, comvi))
    tmp = []
    for i in range(1, comlen):
        results = model1.predict(nfiddats, batch_size=batchsize)
        res2 = model.predict(results)
        for c, (t, o) in enumerate(zip(results, res2)):
            m = np.argmax(t)
            p = np.argmax(o)
            
            nfiddats[comseqpos][c][i] = m
 
    final_data = {}
    final_real = {}
    for fid, com, real in zip(batch.keys(), nfiddats[comseqpos], new_out):
        final_data[fid] = seq2sent(com, comstok)
        final_real[fid] = seq2sent(real, comstok)
        
    for key, val in final_data.items():
        print(key, val)
 
    

In [ ]:
model1.summary()

In [ ]:
d1 = dats1[0].reshape(1,-1)[0]
c1 = coms1[0].reshape(1,-1)[0]
d = np.zeros(len(d1))
c = np.zeros(len(c1))
for i in range(len(d1)):
    d[i] = d1[i]
for i in range(len(c1)):
    c[i] = c1[i]
    
print(d, c)
model1.predict_on_batch([[d], [c]])
    